In [ ]:
!pip install googleDriveFileDownloader

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
import pandas as pd
import numpy as np

In [ ]:
import tensorflow as tf
from skimage import data, exposure, img_as_float
import cv2
from imutils import paths
import random
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import ImageMath
import numpy as np
from sklearn.model_selection import train_test_split
import gc
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import pandas as pd
import zipfile
from sklearn.utils import class_weight

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
gdd.download_file_from_google_drive(file_id='1A-bd5bkoUxMMjjblw_h34oepBzLy2WI4',
                                    dest_path='./final_train.zip',
                                    unzip=True)

In [ ]:
gdd.download_file_from_google_drive(file_id='1fHtcKIox4kEciV8Ti5XvVfVaSaGlhJhG',
                                    dest_path='./final.zip',
                                    unzip=True)

In [ ]:
teste = pd.read_csv('final_train_df.csv')
  
train = pd.read_csv('final_test_df.csv')

In [ ]:
train

In [ ]:
train = train.sample(frac=1).reset_index(drop=True)
teste = teste.sample(frac=1).reset_index(drop=True)

In [ ]:
train.drop(columns= ['imageName'], inplace=True)
teste.drop(columns= ['imageName'], inplace=True)

In [ ]:
train['imagePath'] = train['imagePath'].str.replace('final_train/data/', '')
teste['imagePath'] = teste['imagePath'].str.replace('final_train/data/', '')

In [ ]:
for col in ['epidural', 'intraparenchymal', 'intraventricular',
       'subarachnoid', 'subdural', 'any']:
    train[col] = train[col].astype('float32')
    teste[col] = teste[col].astype('float32')

In [ ]:
data_gen = ImageDataGenerator(rescale = 1/255.)

test_gen = ImageDataGenerator(rescale = 1/255.)

In [ ]:
columns = [ 'epidural', 'intraparenchymal' , 'intraventricular' , 'subarachnoid', 'subdural', 'any']


In [ ]:
train_generator = data_gen.flow_from_dataframe(dataframe=train,
x_col="imagePath",
y_col=columns,
directory = './final_train/data/',
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(224,224))

In [ ]:
test_generator = test_gen.flow_from_dataframe(dataframe=teste,
x_col="imagePath",
y_col=columns,
directory = './final_train/data/',
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(224,224))

In [ ]:
img_shape = (224,224,3)

In [ ]:
# base_model = tf.keras.applications.VGG16(input_shape = img_shape, 
#                                                include_top = False,
#                                                weights = "imagenet")

base_model = tf.keras.applications.InceptionV3(input_shape = img_shape, 
                                               include_top = False,
                                               weights = "imagenet")

# base_model = tf.keras.applications.ResNet50(input_shape = img_shape, 
#                                                include_top = False,
#                                                weights = "imagenet")

In [ ]:
base_model.trainable = False

In [ ]:
base_model.output

In [ ]:
model = tf.keras.Sequential()
model.add(base_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(6, activation='sigmoid'))

In [ ]:
lrate = 0.00001
epochs = 90000

optimizer = tf.keras.optimizers.Adam(learning_rate=lrate)


In [ ]:
checkpoint1 = ModelCheckpoint("./normal-models/houdout_inception_sub_loss.hdf5", monitor='val_loss', verbose=1,
    save_best_only=True, mode='auto', period=1)


In [ ]:
def _weighted_log_loss(y_true, y_pred):
    
    class_weights = np.array([2, 1, 1, 1, 1, 1])

    y_pred = tf.keras.backend.clip(y_pred, tf.keras.backend.epsilon(), 1.0-tf.keras.backend.epsilon())
    out = -(         y_true  * tf.keras.backend.log(      y_pred) * class_weights
            + (1.0 - y_true) * tf.keras.backend.log(1.0 - y_pred) * class_weights)
    
    return tf.keras.backend.mean(out, axis=-1)

In [ ]:
!pip3 install gitpython

In [ ]:
from git import Repo

PATH_OF_GIT_REPO = r'/content/normal-models/.git'  # make sure .git folder is properly configured
COMMIT_MESSAGE = 'update'

def git_push():
    try:
        repo = Repo(PATH_OF_GIT_REPO)
        repo.git.add(update=True)
        repo.index.commit(COMMIT_MESSAGE)
        origin = repo.remote(name='origin')
        origin.push()
    except:
        print('Error')    


In [ ]:
from threading import Timer,Thread,Event

class perpetualTimer():

   def __init__(self,t,hFunction):
      self.t=t
      self.hFunction = hFunction
      self.thread = Timer(self.t,self.handle_function)

   def handle_function(self):
      self.hFunction()
      self.thread = Timer(self.t,self.handle_function)
      self.thread.start()

   def start(self):
      self.thread.start()

   def cancel(self):
      self.thread.cancel()


In [ ]:
stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

In [ ]:
model.compile(optimizer=optimizer, loss = _weighted_log_loss, metrics=['MSE', tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])


In [ ]:
t = perpetualTimer(700, git_push)
t.start()

In [ ]:

history = model.fit(train_generator, validation_data = test_generator, epochs= epochs, verbose=1,callbacks=[checkpoint1, checkpoint2,stop])

In [ ]:
# print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()